<a href="https://colab.research.google.com/github/yeonin1707/cp2/blob/main/%EC%84%B9%EC%85%98%EB%9D%BC%EB%B2%A8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# Surprise 설치
!pip install scikit-surprise
import warnings
import surprise
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.dataset import DatasetAutoFolds

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 7.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633984 sha256=35ef993d7356647955979170036618a331b3c9e83c2bed9ba6333c3c14104eb8
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/cp2.csv', sep=',')

In [5]:
df

,event_type,product_id,category_id,brand,price,user_id,user_session,date,time,first_level,secont_level,third_level
0,view,3900821,2053013552326770905,aqua,33.20,554748717,57-6336556b0fc,2019-10-01,00:00:00,appliances,environment,water_heater
1,view,1480613,2053013561092866779,pulser,908.62,512742880,a5-86594dec0db,2019-10-01,00:00:05,computers,desktop,desktop
2,view,31500053,2053013558031024687,luminarc,41.16,550978835,a7-abc6048498d,2019-10-01,00:00:08,empty,empty,empty
3,view,2900536,2053013554776244595,elenberg,51.46,555158050,9e-9ce44bb50ab,2019-10-01,00:00:11,appliances,kitchen,microwave
4,view,1480714,2053013561092866779,pulser,921.49,512742880,a5-86594dec0db,2019-10-01,00:00:22,computers,desktop,desktop
...,...,...,...,...,...,...,...,...,...,...,...,...
14052485,view,1004708,2053013555631882655,huawei,151.99,566031393,5c-cee9c52ab9a,2019-10-31,23:59:53,electronics,smartphone,smartphone
14052486,view,1480709,2053013561092866779,pulser,592.01,543124499,25-9a0d43297f9,2019-10-31,23:59:55,computers,desktop,desktop
14052487,view,5800263,2053013553945772349,alphard,92.67,551943550,3c-e04aa3163bb,2019-10-31,23:59:55,electronics,audio,subwoofer
14052488,view,1005008,2053013555631882655,xiaomi,94.46,557202408,61-ac70ff0c66e,2019-10-31,23:59:56,electronics,smartphone,smartphone


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14052490 entries, 0 to 14052489
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   event_type    object 
 1   product_id    int64  
 2   category_id   int64  
 3   brand         object 
 4   price         float64
 5   user_id       int64  
 6   user_session  object 
 7   date          object 
 8   time          object 
 9   first_level   object 
 10  secont_level  object 
 11  third_level   object 
dtypes: float64(1), int64(3), object(8)
memory usage: 1.3+ GB


***데이터 조합***

In [8]:
# 대분류의 카테고리 개수 14
print(df['first_level'].nunique())
df['first_level'].value_counts().index

14


Index(['electronics', 'empty', 'appliances', 'computers', 'apparel',
       'furniture', 'auto', 'construction', 'kids', 'accessories', 'sport',
       'medicine', 'country_yard', 'stationery'],
      dtype='object')

In [9]:
# 중분류의 카테고리의 개수 58
print(df['secont_level'].nunique())
df['secont_level'].value_counts().index

58


Index(['empty', 'smartphone', 'kitchen', 'audio', 'clocks', 'shoes',
       'environment', 'notebook', 'video', 'accessories', 'tools',
       'living_room', 'desktop', 'bedroom', 'peripherals', 'components',
       'tablet', 'telephone', 'carriage', 'bag', 'iron', 'sewing_machine',
       'personal', 'costume', 'bicycle', 'toys', 'skates', 'jeans', 'camera',
       'dolls', 'underwear', 'wallet', 'dress', 'trainer', 'ironing_board',
       'ebooks', 'swing', 'bathroom', 'fmcg', 'tshirt', 'trousers', 'shirt',
       'universal', 'cultivator', 'cartrige', 'lawn_mower', 'umbrella',
       'tennis', 'snowboard', 'ski', 'jumper', 'sock', 'belt', 'scarf',
       'furniture', 'skirt', 'shorts', 'jacket'],
      dtype='object')

In [10]:
df.groupby('first_level').count()

,event_type,product_id,category_id,brand,price,user_id,user_session,date,time,secont_level,third_level
first_level,,,,,,,,,,,
accessories,78980,78980,78980,78980,78980,78980,78980,78980,78980,78980,78980
apparel,505639,505639,505639,505639,505639,505639,505639,505639,505639,505639,505639
appliances,1654681,1654681,1654681,1654681,1654681,1654681,1654681,1654681,1654681,1654681,1654681
auto,328676,328676,328676,328676,328676,328676,328676,328676,328676,328676,328676
computers,769235,769235,769235,769235,769235,769235,769235,769235,769235,769235,769235
construction,241824,241824,241824,241824,241824,241824,241824,241824,241824,241824,241824
country_yard,4479,4479,4479,4479,4479,4479,4479,4479,4479,4479,4479
electronics,5338748,5338748,5338748,5338748,5338748,5338748,5338748,5338748,5338748,5338748,5338748
empty,4477560,4477560,4477560,4477560,4477560,4477560,4477560,4477560,4477560,4477560,4477560


In [11]:
# 대분류의 유일값을 라벨링 하기 

#리스트로 유일값 만들기 
df['first_level'].unique().tolist()
#인덱스에 리턴
df['first_level'].unique().tolist().index('appliances')
#만든 리스트를 변수저장
temp_list_1 = df['first_level'].unique().tolist()
#람다함수 사용해서 라벨링 하기 
df['first_level'].map(lambda x: temp_list_1.index(x))+1

0           1
1           2
2           3
3           1
4           2
           ..
14052485    4
14052486    2
14052487    4
14052488    4
14052489    3
Name: first_level, Length: 14052490, dtype: int64

In [12]:
df.insert(9, 'level_1', df['first_level'].map(lambda x: temp_list_1.index(x)))
df.tail()

,event_type,product_id,category_id,brand,price,user_id,user_session,date,time,level_1,first_level,secont_level,third_level
14052485,view,1004708,2053013555631882655,huawei,151.99,566031393,5c-cee9c52ab9a,2019-10-31,23:59:53,3,electronics,smartphone,smartphone
14052486,view,1480709,2053013561092866779,pulser,592.01,543124499,25-9a0d43297f9,2019-10-31,23:59:55,1,computers,desktop,desktop
14052487,view,5800263,2053013553945772349,alphard,92.67,551943550,3c-e04aa3163bb,2019-10-31,23:59:55,3,electronics,audio,subwoofer
14052488,view,1005008,2053013555631882655,xiaomi,94.46,557202408,61-ac70ff0c66e,2019-10-31,23:59:56,3,electronics,smartphone,smartphone
14052489,view,5100816,2053013553375346967,xiaomi,29.58,543653226,5b-21ab2010925,2019-10-31,23:59:58,2,empty,empty,empty


In [13]:
# 중분류의 유일값을 라벨링 하기 

#리스트로 유일값 만들기 
df['secont_level'].unique().tolist()
#인덱스에 리턴
df['secont_level'].unique().tolist().index('environment')
#만든 리스트를 변수저장
temp_list_2 = df['secont_level'].unique().tolist()
#람다함수 사용해서 라벨링 하기 
df['secont_level'].map(lambda x: temp_list_2.index(x))


0            0
1            1
2            2
3            3
4            1
            ..
14052485     5
14052486     1
14052487    14
14052488     5
14052489     2
Name: secont_level, Length: 14052490, dtype: int64

In [14]:
df.insert(9, 'level_2', df['secont_level'].map(lambda x: temp_list_2.index(x)))
df.tail()

,event_type,product_id,category_id,brand,price,user_id,user_session,date,time,level_2,level_1,first_level,secont_level,third_level
14052485,view,1004708,2053013555631882655,huawei,151.99,566031393,5c-cee9c52ab9a,2019-10-31,23:59:53,5,3,electronics,smartphone,smartphone
14052486,view,1480709,2053013561092866779,pulser,592.01,543124499,25-9a0d43297f9,2019-10-31,23:59:55,1,1,computers,desktop,desktop
14052487,view,5800263,2053013553945772349,alphard,92.67,551943550,3c-e04aa3163bb,2019-10-31,23:59:55,14,3,electronics,audio,subwoofer
14052488,view,1005008,2053013555631882655,xiaomi,94.46,557202408,61-ac70ff0c66e,2019-10-31,23:59:56,5,3,electronics,smartphone,smartphone
14052489,view,5100816,2053013553375346967,xiaomi,29.58,543653226,5b-21ab2010925,2019-10-31,23:59:58,2,2,empty,empty,empty


In [15]:
##유저섹션 유니크값 30만개
print(df['user_session'].nunique())
df['user_session'].value_counts().index

3064625


Index(['8a-1fadfc58375', '84-427eaa9fabf', 'bc-b39c66dcd0b', 'e5-b9c983361ef',
       '52-efab8257d37', '43-b7e5192cbd7', '87-7f6dfdaf292', '00-1e7e1ac7f4e',
       '54-00d41932871', '3f-31ee98a3285',
       ...
       '85-26ebe2536ad', 'bf-7240786d9e3', 'e9-ff22a5d4f26', '14-fdc4ee17e81',
       '87-97d90279db2', 'ce-e5bb718480c', '01-6738280ee4e', '07-2e7acfbdca4',
       'cb-19fea234e46', '61-ac70ff0c66e'],
      dtype='object', length=3064625)

In [ ]:
#리스트로 유일값 만들기 
df['user_session'].unique().tolist()
#인덱스에 리턴
df['user_session'].unique().tolist().index('8a-1fadfc58375')
#만든 리스트를 변수저장
temp_list_2 = df['user_session'].unique().tolist()
#람다함수 사용해서 라벨링 하기 
df['user_session'].map(lambda x: temp_list_2.index(x))

In [ ]:
df.to_csv('df1.csv',index=False)